# Load the Dataset

In [8]:
from datasets import load_dataset
from rich import print

faq_dataset = load_dataset("csv" , data_files="/home/null/code/utd_chatbot/FAQ on UTD Student Accesebility - Sheet1.csv")
print(faq_dataset)
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)
## reaplace all the word ARC to AccessAbility Resource Center and
'''
Office location: Administration Building, Room 2.224
Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.)
Phone: (972) 883-2098
Fax: Please don’t fax, use utd.link/arcupload
Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080
'''
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('ARC', 'AccessAbility Resource Center'), 'Answering': x['Answering'].replace('ARC', 'AccessAbility Resource Center')})

/home/null/miniconda3/envs/utd_chatbot/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-b5279144beba40c5
Found cached dataset csv (/home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 512.75it/s]


DatasetDict({
    train: Dataset({
        features: ['Question', 'Answering', 'URL', 'Label'],
        num_rows: 23
    })
})

Loading cached processed dataset at /home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-d8b863d6763fbde1.arrow
100%|██████████| 20/20 [00:00<00:00, 4033.57ex/s]


# Preprocess the dataset

## remove all the None values

In [9]:
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)

100%|██████████| 1/1 [00:00<00:00, 55.15ba/s]


## Replace all the ARC to Accessible Resource Center

In [10]:
# replace all the word UTD to University of Texas at Dallas
# replace all the ARC to AccessAbility Resource Center
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('UTD', 'University of Texas at Dallas'), 'Answering': x['Answering'].replace('UTD', 'University of Texas at Dallas')})
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('ARC', 'AccessAbility Resource Center'), 'Answering': x['Answering'].replace('ARC', 'AccessAbility Resource Center')})

100%|██████████| 20/20 [00:00<00:00, 5174.32ex/s]


# Load the Open AI Emebddigns

In [11]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.

OPENKEY_API = os.getenv("OPENAI_KEY")
ORGANIZATION_ID = os.getenv("ORGANIZATION_ID")
openai.organization = ORGANIZATION_ID
# get this from top-right dropdown on OpenAI under organization > settings
openai.api_key = OPENKEY_API
# get API key from top-right dropdown on OpenAI website

openai.Engine.list()  # check we have authenticated
## model of choices
MODEL = "text-similarity-babbage-001"

# Ini the Pinecone Client

In [12]:
import pinecone
PINECONE_APIKEY = os.getenv("PINECONE_APIKEY")
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key = PINECONE_APIKEY,
    environment="us-east1-gcp"
)

In [13]:
index = pinecone.Index(index_name="utd-chabot")

# Query a Vector Database

In [14]:
def query_vector_database(query):
    xq = openai.Embedding.create(input=query, engine=MODEL)['data'][0]['embedding']
    res = index.query([xq], top_k=5, include_metadata=True)
    return res

# Role Prompt

In [15]:
role_prompt = """
Please act as a University of Texas at Dallas Counselor. I will provide you with an individual 
looking for guidance at the University of Texas at Dallas, and your task is to help them 
solve their problem\n
"""

# Create an standard QA Propmt

In [16]:
def create_an_standard_qa_prompt( query , res):
    import copy
    role_prompt_temp =  copy.deepcopy(role_prompt)
    for query_response in  res["matches"]:
        answer = faq_dataset['train'].filter(lambda x: x['Question'] == query_response['metadata']['text'])['Answering'][0]
        role_prompt_temp += f"Q: {query_response['metadata']['text']}\nA: {answer}\n"
    prompt = role_prompt_temp + f"Q: {query}\nA:Let’s think step by step."
    return prompt

# Called a LLM Decoder Model

In [ ]:
def called_llm_decoder_model(prompt):
  response = openai.Completion.create(
    model="text-davinci-002",
    prompt =  prompt,
    temperature = 0.8,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response["choices"][0]["text"]

# Post Process of the Output

In [ ]:
def post_process(ans):
    if "ARC" in ans:
        ans = ans.replace("ARC", "AccessAbility Resource Center")
        ans += """ 
        Office location: Administration Building, Room 2.224 \n
        Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.) \n
        Phone: (972) 883-2098 \n
        Fax: Please don’t fax, use utd.link/arcupload \n
        Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080    Office location: Administration Building, Room 2.224\n
        """
    return ans

# Gradio

In [ ]:
import gradio as gr
questions = []

def utd_chatbot( question ):
    questions.append( question )
    ## store the question in the csvs file
    import pandas as pd
    data = pd.DataFrame(question)
    data.to_csv("questions.csv")
    propmt = create_an_standard_qa_prompt(question, query_vector_database(question))
    ans = called_llm_decoder_model(propmt)
    post_process(ans)
    return ans

demo = gr.Interface(fn=utd_chatbot, inputs="text", outputs="text")

demo.launch()   

The first thing you need to do is get documentation from a qualified professional (e.g. physician, psychologist, 
licensed counselor, etc.) that indicates a diagnosis of ADHD and how it impacts your academic performance. This 
documentation is important because it will help determine what accommodations may be appropriate for you. 

Once you have documentation, you will need to make an appointment with the AccessAbility Resource Center to 
register for services and discuss your educational needs. During this appointment, you will also have the 
opportunity to learn more about ADHD and how it may impact your academic success. 

The AccessAbility Resource Center staff will work with you to develop an individualized service plan that includes 
academic accommodations and other support services. Some of the academic accommodations that may be appropriate for
students with ADHD include extended time for test taking, note takers, printed materials in alternate formats, sign
language interpreters and the use of assistive technology.
 
    Office location: Administration Building, Room 2.224 

    Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.) 

    Phone: (972) 883-2098 

    Fax: Please don’t fax, use utd.link/arcupload 

    Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080    Office location: Administration Building, Room 2.224

In [ ]:
import gradio as gr
question = []

def greet(name):
    question.append(name)
    ## store the question in the csvs file
    import pandas as pd
    data = pd.DataFrame(question)
    data.to_csv("questions.csv")
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch()   

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
